# Sentence Emotion Detection Model

This notebook contains code we used to train our model that uses embedding and LSTM sentiment analysis to predict the emotion of a journal entry (text sentence)

## Preperation

Install SpaCy and import relevant libraries


In [1]:
!pip install --upgrade torch==1.7.1 torchtext==0.8.1 torchvision==0.8.2

     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 7.0MB 23.9MB/s 
     |████████████████████████████████| 12.8MB 240kB/s 
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 1.6MB 18.2MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (56.1.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 69 kB/s 
     |████████████████████████████████| 1.1 MB 70.1 MB/s 
     |████████████████████████████████| 9.1 MB 75.6 MB/s 
     |████████████████████████████████| 456 kB 67.6 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 113 kB 79.5 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=2386f5914fcc8567666ad5367511b8a814e290c8ecbc5f915bea36ffc94e86fe
  Stored in directory: /root/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attem

In [3]:
import torch, torchtext
from torch import nn, optim, functional as F
import pandas as pd, csv
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
import random

Import dataset (already cleaned) from dropbox link

In [4]:
!wget -O text.csv https://www.dropbox.com/s/iulhdbo1yc8farq/Emotion_final.csv?dl=0

--2021-05-08 02:38:39--  https://www.dropbox.com/s/iulhdbo1yc8farq/Emotion_final.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/iulhdbo1yc8farq/Emotion_final.csv [following]
--2021-05-08 02:38:39--  https://www.dropbox.com/s/raw/iulhdbo1yc8farq/Emotion_final.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdc3fdedc4f89114ffbb151d3ec.dl.dropboxusercontent.com/cd/0/inline/BOHGr4Tzo_jybb9jKqCA6sCthVlc7iMSTkonCaBRn-jBa_uaza65sX3RHnFpkst_TRrPzm7kDRQirX4GL_rN2t-cC2ua9bUCZXQv3L6PhJNvLSsuLuTDof5ai4DQPJQYRfEPijvknunGKqoLZRDgCYhq/file# [following]
--2021-05-08 02:38:39--  https://ucdc3fdedc4f89114ffbb151d3ec.dl.dropboxusercontent.com/cd/0/inline/BOHGr4Tzo_jybb9jKqCA6sCthVlc7iMSTkonCaBRn-jBa_uaza65sX3RHnFpkst_TRrPzm

In [5]:
text = pd.read_csv('/content/text.csv')

In [ ]:
text

,Number,Text,Emotion
0,1,i didnt feel humiliated,sadness
1,2,i can go from feeling so hopeless to so damned...,sadness
2,3,im grabbing a minute to post i feel greedy wrong,anger
3,4,i am ever feeling nostalgic about the fireplac...,love
4,5,i am feeling grouchy,anger
...,...,...,...
21454,21455,Melissa stared at her friend in dism,fear
21455,21456,Successive state elections have seen the gover...,fear
21456,21457,Vincent was irritated but not dismay,fear
21457,21458,Kendall-Hume turned back to face the dismayed ...,fear


Sentiments into an array for later use

In [ ]:
text.Emotion.unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'happy'],
      dtype=object)

In [6]:
sentiment = ['sadness', 'anger', 'love', 'surprise', 'fear', 'happy']

## Dataset

Define Dataset for text and split into train/test subsets

In [7]:
class Sentences(torch.utils.data.Dataset):
    def __init__(self, fn):
        lengths = []
        convert = { u: n for n, u in enumerate(fn['Emotion'].unique()) }
        fn['Emotion'] = fn['Emotion'].apply(lambda u: convert[u])               # 12 unique words should be assigned integers starting from 0
        tokenizer = torchtext.data.utils.get_tokenizer('spacy', 'en_core_web_sm')# tokenizer using spaCy
        for i in range(len(fn['Text'])):
          lengths.append(len(tokenizer(fn['Text'].iat[i].strip())))                   # store the number of tokens in each sentence to beused in get item
        string = ' '.join([fn['Text'].iat[i].strip() 
                           for i in range(len(fn['Text']))])                  # combine everything into one single string
        toks = tokenizer(string)                                                # tokenize the single string

        self.vocab = torchtext.vocab.build_vocab_from_iterator([toks])
        self.sentiment = fn['Emotion'].values
        self.text = fn['Text'].values
        self.length = lengths
        self.toks = torch.LongTensor([self.vocab[tok] for tok in toks])

    def __len__(self):
        return len(self.length)

    def __getitem__(self, i):
        sum = 0
        for x in range(i):
          sum += self.length[x]
        return (self.sentiment[i], self.toks[sum: sum + self.length[i]])          # return the sentiment and related tokns for a specific tweet

In [8]:
ds_full = Sentences(text)
n_train = int(0.8 * len(ds_full))
n_test = len(ds_full) - n_train
rng = torch.Generator().manual_seed(291)
ds_train, ds_test = torch.utils.data.random_split(ds_full, [n_train, n_test], rng)

1lines [00:00, 21.18lines/s]


Check outputs if the outputs are what we expect (tensor with integer corresponding to label and tensor of integers corresponding to tokens which can be converted to a sentence)

In [ ]:
print(ds_full[100])

In [ ]:
print(ds_full[100][0])

In [ ]:
sentiment[ds_test[100][0]]

In [ ]:
print(' '.join([ds_full.vocab.itos[x] for x in ds_full[100][1]]))

In [ ]:
len(ds_full.toks)

## Model

Model with embedding and LSTM

In [9]:
class SentenceModel(nn.Module):                                                 # takes in a sentence, and outputs predicted sentiment
      def __init__(self, vocab_size, embedding_dim, lstm_dim, 
                   n_cats, n_layers = 2, drop_prob = 0.5):
        super().__init__()                                                      #constructor for parent class
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)          #use word embeddings 
        self.lstm = torch.nn.LSTM(embedding_dim, lstm_dim, n_layers,
                                  dropout=drop_prob, batch_first=True)          #LSTM layer
        self.linear = nn.Linear(lstm_dim, n_cats)
        nn.init.xavier_uniform_(self.embedding.weight.data)
        nn.init.xavier_uniform_(self.linear.weight.data)
        
      def forward(self, text):
        emb = self.embedding(text)
        lstm_out, _ = self.lstm(emb)
        out = self.linear(lstm_out)
        return torch.mean(out, dim=1)                                           # certain dimensions required so take mean to reduce them down

Test and Train loops

In [10]:
device = torch.device('cpu')

def run_test(model, ds, crit):
    preds = []                                                                  # array to store predictions
    batch_size = 1                                                              # change batch size here
    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds)
    for labs, txts in ldr:                                                
        labs, txts = labs.to(device), txts.to(device)
        with torch.no_grad():
            outs = model(txts)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
            preds.append(outs.argmax(1))                                        # append all the predictions to an array
    return total_loss / len(ds), total_acc / len(ds), preds, batch_size         # added array return value 'preds' and batchsize

def run_train(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds)
    for labs, txts in ldr:          
        opt.zero_grad()
        labs, txts = labs.to(device), txts.to(device)
        outs = model(txts)                                                      
        loss = crit(outs, labs)
        loss.backward()
        opt.step()
        total_loss += loss.item()
        total_acc += (outs.argmax(1) == labs).sum().item()
    sched.step()
    return total_loss / len(ds), total_acc / len(ds)


def run_all(model, test_ds, train_ds, crit, opt, sched, n_epochs=10, early_stop=False):
    max_test_acc = 0;
    for epoch in tqdm(range(n_epochs), desc='epochs'):
        train_loss, train_acc = run_train(model, train_ds, crit, opt, sched)
        test_loss, test_acc, _, _ = run_test(model, test_ds, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f} acc {train_acc:.4f}   test loss {test_loss:.6f} acc {test_acc:.4f}')  
        if (early_stop): 
          if (test_acc >= max_test_acc):
            max_test_acc = test_acc
          else:
            print("EARLY STOPPED")
            break

## Training

Train model by adjusting the hyperparameters (optimizer, scheduler, learning rate, step size, gamma, dimensions etc.) to improve the model's test accuracy

In [ ]:
#TEST 1

model = SentenceModel(len(ds_full.vocab), 32, 1, len(text.Emotion.unique()))
device = torch.device('cuda:0') #added GPU since CPU too slow (enable that in notebook settings)
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 10, gamma=0.1)

run_all(model, ds_test, ds_train, crit, opt, sched, 10)

epoch 0   train loss 1.809199 acc 0.2762   test loss 1.712606 acc 0.3269
epoch 1   train loss 1.809170 acc 0.2764   test loss 1.712604 acc 0.3269
epoch 2   train loss 1.809170 acc 0.2764   test loss 1.712607 acc 0.3269
epoch 3   train loss 1.809170 acc 0.2764   test loss 1.712606 acc 0.3269
epoch 4   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269
epoch 5   train loss 1.809170 acc 0.2764   test loss 1.712606 acc 0.3269
epoch 6   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269
epoch 7   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269
epoch 8   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269
epoch 9   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269



In [ ]:
#TEST 2

model = SentenceModel(len(ds_full.vocab), 32, 1, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.1) #step size: 10->1

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809194 acc 0.2763   test loss 1.712609 acc 0.3269
epoch 1   train loss 1.606745 acc 0.3163   test loss 1.583358 acc 0.3269
epoch 2   train loss 1.590632 acc 0.3243   test loss 1.576448 acc 0.3269
epoch 3   train loss 1.588923 acc 0.3277   test loss 1.575377 acc 0.3269
epoch 4   train loss 1.588439 acc 0.3277   test loss 1.575337 acc 0.3269
epoch 5   train loss 1.588368 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 6   train loss 1.588358 acc 0.3277   test loss 1.575332 acc 0.3269
epoch 7   train loss 1.588357 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 8   train loss 1.588359 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 9   train loss 1.588360 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 10   train loss 1.588359 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 11   train loss 1.588359 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 12   train loss 1.588360 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 13   train loss 1.588360 acc 0.3277   test

In [ ]:
#TEST 3

model = SentenceModel(len(ds_full.vocab), 32, 64, len(text.Emotion.unique())) #lstm_dim: 1 -> 64
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809693 acc 0.2763   test loss 1.712638 acc 0.3269
epoch 1   train loss 1.606746 acc 0.3163   test loss 1.583356 acc 0.3269
epoch 2   train loss 1.590628 acc 0.3243   test loss 1.576445 acc 0.3269
epoch 3   train loss 1.588917 acc 0.3277   test loss 1.575373 acc 0.3269
epoch 4   train loss 1.588433 acc 0.3277   test loss 1.575333 acc 0.3269
epoch 5   train loss 1.588362 acc 0.3277   test loss 1.575330 acc 0.3269
epoch 6   train loss 1.588352 acc 0.3277   test loss 1.575327 acc 0.3269
epoch 7   train loss 1.588349 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 8   train loss 1.588348 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 9   train loss 1.588348 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 10   train loss 1.588348 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 11   train loss 1.588349 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 12   train loss 1.588348 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 13   train loss 1.588348 acc 0.3277   test

In [ ]:
#TEST 4

model = SentenceModel(len(ds_full.vocab), 32, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1) #gamma: 0.1 -> 1

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809704 acc 0.2762   test loss 1.712623 acc 0.3269
epoch 1   train loss 1.809252 acc 0.2764   test loss 1.712635 acc 0.3269
epoch 2   train loss 1.809233 acc 0.2764   test loss 1.712613 acc 0.3269
epoch 3   train loss 1.809228 acc 0.2765   test loss 1.712626 acc 0.3269
epoch 4   train loss 1.809226 acc 0.2765   test loss 1.712626 acc 0.3269
epoch 5   train loss 1.809223 acc 0.2765   test loss 1.712630 acc 0.3269
epoch 6   train loss 1.809220 acc 0.2765   test loss 1.712604 acc 0.3269
epoch 7   train loss 1.809220 acc 0.2764   test loss 1.712621 acc 0.3269
epoch 8   train loss 1.809219 acc 0.2764   test loss 1.712647 acc 0.3269
epoch 9   train loss 1.809215 acc 0.2765   test loss 1.712696 acc 0.3269
epoch 10   train loss 1.809222 acc 0.2764   test loss 1.712615 acc 0.3269
epoch 11   train loss 1.809224 acc 0.2764   test loss 1.712678 acc 0.3269
epoch 12   train loss 1.809221 acc 0.2765   test loss 1.712667 acc 0.3269
epoch 13   train loss 1.809232 acc 0.2764   test

In [ ]:
#TEST 5

model = SentenceModel(len(ds_full.vocab), 32, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.0001) #gamma: 1 -> 0.0001

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809709 acc 0.2762   test loss 1.712674 acc 0.3269
epoch 1   train loss 1.685513 acc 0.3277   test loss 1.633857 acc 0.3269
epoch 2   train loss 1.652987 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 3   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 4   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 5   train loss 1.652985 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 6   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 7   train loss 1.652988 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 8   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 9   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 10   train loss 1.652987 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 11   train loss 1.652987 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 12   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 13   train loss 1.652985 acc 0.3277   test

In [ ]:
#TEST 6

model = SentenceModel(len(ds_full.vocab), 32, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=3.0) #lr: 1.0 -> 3.0
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 2.561450 acc 0.2328   test loss 2.465432 acc 0.3269
epoch 1   train loss 2.550762 acc 0.2328   test loss 2.547255 acc 0.3269
epoch 2   train loss 2.550788 acc 0.2328   test loss 2.470872 acc 0.3269
epoch 3   train loss 2.551229 acc 0.2328   test loss 2.464573 acc 0.3269
epoch 4   train loss 2.550795 acc 0.2328   test loss 2.462552 acc 0.3269
epoch 5   train loss 2.550712 acc 0.2328   test loss 2.472194 acc 0.3269
epoch 6   train loss 2.550416 acc 0.2327   test loss 2.463349 acc 0.3269
epoch 7   train loss 2.552428 acc 0.2329   test loss 2.464066 acc 0.3269
epoch 8   train loss 2.550916 acc 0.2328   test loss 2.472926 acc 0.3269
epoch 9   train loss 2.550116 acc 0.2328   test loss 2.499187 acc 0.3269
epoch 10   train loss 2.549954 acc 0.2328   test loss 2.501174 acc 0.3269
epoch 11   train loss 2.550029 acc 0.2328   test loss 2.502170 acc 0.3269
epoch 12   train loss 2.549956 acc 0.2329   test loss 2.501480 acc 0.3269
epoch 13   train loss 2.550230 acc 0.2329   test

In [ ]:
#TEST 7

model = SentenceModel(len(ds_full.vocab), 32, 128, len(text.Emotion.unique())) #lstm_dim: 64 -> 128
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809891 acc 0.2763   test loss 1.712609 acc 0.3269
epoch 1   train loss 1.809264 acc 0.2765   test loss 1.712634 acc 0.3269
epoch 2   train loss 1.809246 acc 0.2765   test loss 1.712635 acc 0.3269
epoch 3   train loss 1.809242 acc 0.2764   test loss 1.712628 acc 0.3269
epoch 4   train loss 1.809242 acc 0.2765   test loss 1.712622 acc 0.3269
epoch 5   train loss 1.809221 acc 0.2765   test loss 1.712653 acc 0.3269
epoch 6   train loss 1.809227 acc 0.2764   test loss 1.712622 acc 0.3269
epoch 7   train loss 1.809225 acc 0.2764   test loss 1.712643 acc 0.3269
epoch 8   train loss 1.809215 acc 0.2765   test loss 1.712636 acc 0.3269
epoch 9   train loss 1.809212 acc 0.2765   test loss 1.712626 acc 0.3269
epoch 10   train loss 1.809205 acc 0.2764   test loss 1.712622 acc 0.3269
epoch 11   train loss 1.809205 acc 0.2765   test loss 1.712624 acc 0.3269
epoch 12   train loss 1.809210 acc 0.2765   test loss 1.712615 acc 0.3269
epoch 13   train loss 1.809207 acc 0.2764   test

In [ ]:
#TEST 8

model = SentenceModel(len(ds_full.vocab), 16, 64, len(text.Emotion.unique())) #embedding_dim: 32 -> 16
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809728 acc 0.2763   test loss 1.712628 acc 0.3269
epoch 1   train loss 1.809250 acc 0.2765   test loss 1.712639 acc 0.3269
epoch 2   train loss 1.809241 acc 0.2765   test loss 1.712644 acc 0.3269
epoch 3   train loss 1.809226 acc 0.2765   test loss 1.712641 acc 0.3269
epoch 4   train loss 1.809217 acc 0.2765   test loss 1.712638 acc 0.3269
epoch 5   train loss 1.809213 acc 0.2765   test loss 1.712620 acc 0.3269
epoch 6   train loss 1.809217 acc 0.2765   test loss 1.712643 acc 0.3269
epoch 7   train loss 1.809215 acc 0.2765   test loss 1.712664 acc 0.3269
epoch 8   train loss 1.809216 acc 0.2764   test loss 1.712653 acc 0.3269
epoch 9   train loss 1.809217 acc 0.2765   test loss 1.712630 acc 0.3269
epoch 10   train loss 1.809216 acc 0.2764   test loss 1.712659 acc 0.3269
epoch 11   train loss 1.809215 acc 0.2764   test loss 1.712645 acc 0.3269
epoch 12   train loss 1.809215 acc 0.2765   test loss 1.712651 acc 0.3269
epoch 13   train loss 1.809215 acc 0.2764   test

In [ ]:
#TEST 9

model = SentenceModel(len(ds_full.vocab), 64, 64, len(text.Emotion.unique())) #embedding_dim: 16 -> 64
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 30)

epoch 0   train loss 1.809703 acc 0.2763   test loss 1.712663 acc 0.3269
epoch 1   train loss 1.809243 acc 0.2765   test loss 1.712636 acc 0.3269
epoch 2   train loss 1.809228 acc 0.2764   test loss 1.712638 acc 0.3269
epoch 3   train loss 1.809219 acc 0.2764   test loss 1.712617 acc 0.3269
epoch 4   train loss 1.809223 acc 0.2765   test loss 1.712676 acc 0.3269
epoch 5   train loss 1.809223 acc 0.2765   test loss 1.712635 acc 0.3269
epoch 6   train loss 1.809218 acc 0.2765   test loss 1.712621 acc 0.3269
epoch 7   train loss 1.809225 acc 0.2764   test loss 1.712636 acc 0.3269
epoch 8   train loss 1.809224 acc 0.2765   test loss 1.712636 acc 0.3269
epoch 9   train loss 1.809228 acc 0.2764   test loss 1.712652 acc 0.3269
epoch 10   train loss 1.809223 acc 0.2765   test loss 1.712687 acc 0.3269
epoch 11   train loss 1.809230 acc 0.2764   test loss 1.712692 acc 0.3269
epoch 12   train loss 1.809240 acc 0.2764   test loss 1.712769 acc 0.3269
epoch 13   train loss 1.809255 acc 0.2762   test

In [ ]:
#TEST 10

model = SentenceModel(len(ds_full.vocab), 128, 64, len(text.Emotion.unique())) #embedding_dim: 64 -> 128
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 30)

epoch 0   train loss 1.809674 acc 0.2761   test loss 1.712636 acc 0.3269
epoch 1   train loss 1.809237 acc 0.2765   test loss 1.712614 acc 0.3269
epoch 2   train loss 1.809224 acc 0.2765   test loss 1.712659 acc 0.3269
epoch 3   train loss 1.809239 acc 0.2765   test loss 1.712636 acc 0.3269
epoch 4   train loss 1.809243 acc 0.2764   test loss 1.712636 acc 0.3269
epoch 5   train loss 1.809233 acc 0.2763   test loss 1.712630 acc 0.3269
epoch 6   train loss 1.809230 acc 0.2764   test loss 1.712678 acc 0.3269
epoch 7   train loss 1.809235 acc 0.2764   test loss 1.712655 acc 0.3269
epoch 8   train loss 1.809245 acc 0.2763   test loss 1.712886 acc 0.3269
epoch 9   train loss 1.809171 acc 0.2759   test loss 1.766219 acc 0.3269
epoch 10   train loss 1.389631 acc 0.4698   test loss 0.875569 acc 0.6980
epoch 11   train loss 1.058079 acc 0.6326   test loss 0.895942 acc 0.7046
epoch 12   train loss 1.154795 acc 0.5970   test loss 1.035404 acc 0.6636
epoch 13   train loss 1.175448 acc 0.5947   test

In [ ]:
#TEST 11

model = SentenceModel(len(ds_full.vocab), 64, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=0.1) #lr: 1 -> 0.1 (counteract the loss increase over time)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.608359 acc 0.3168   test loss 1.583686 acc 0.3269
epoch 1   train loss 1.606533 acc 0.3162   test loss 1.583427 acc 0.3269
epoch 2   train loss 1.606486 acc 0.3162   test loss 1.583368 acc 0.3269
epoch 3   train loss 1.606475 acc 0.3161   test loss 1.583344 acc 0.3269
epoch 4   train loss 1.606457 acc 0.3162   test loss 1.583283 acc 0.3269
epoch 5   train loss 1.606383 acc 0.3161   test loss 1.583106 acc 0.3269
epoch 6   train loss 1.605800 acc 0.3163   test loss 1.582347 acc 0.3269
epoch 7   train loss 1.580952 acc 0.3430   test loss 1.643022 acc 0.4007
epoch 8   train loss 1.259339 acc 0.5179   test loss 1.073486 acc 0.5874
epoch 9   train loss 0.934875 acc 0.6407   test loss 0.759820 acc 0.7216
epoch 10   train loss 0.656265 acc 0.7604   test loss 0.521467 acc 0.8197
epoch 11   train loss 0.425697 acc 0.8469   test loss 0.382146 acc 0.8681
epoch 12   train loss 0.303684 acc 0.8896   test loss 0.297928 acc 0.8947
epoch 13   train loss 0.227618 acc 0.9138   test

In [ ]:
#TEST 12

model = SentenceModel(len(ds_full.vocab), 64, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=0.001) #lr: 0.1 -> 0.001
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 30)

epoch 0   train loss 1.609920 acc 0.3241   test loss 1.576558 acc 0.3269
epoch 1   train loss 1.589516 acc 0.3286   test loss 1.576105 acc 0.3269
epoch 2   train loss 1.589294 acc 0.3280   test loss 1.575990 acc 0.3269
epoch 3   train loss 1.589246 acc 0.3285   test loss 1.575897 acc 0.3269
epoch 4   train loss 1.589119 acc 0.3289   test loss 1.575800 acc 0.3269
epoch 5   train loss 1.589058 acc 0.3289   test loss 1.575715 acc 0.3269
epoch 6   train loss 1.588828 acc 0.3291   test loss 1.575638 acc 0.3267
epoch 7   train loss 1.588867 acc 0.3282   test loss 1.575563 acc 0.3267
epoch 8   train loss 1.588692 acc 0.3292   test loss 1.575492 acc 0.3267
epoch 9   train loss 1.588694 acc 0.3285   test loss 1.575433 acc 0.3271
epoch 10   train loss 1.588491 acc 0.3293   test loss 1.575371 acc 0.3271
epoch 11   train loss 1.588492 acc 0.3291   test loss 1.575310 acc 0.3271
epoch 12   train loss 1.588316 acc 0.3302   test loss 1.575258 acc 0.3271
epoch 13   train loss 1.588284 acc 0.3301   test

In [ ]:
#TEST 13

model = SentenceModel(len(ds_full.vocab), 64, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.Adagrad(model.parameters(), lr=0.1) #optimizer: SGD -> Adagrad
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 30)

epoch 0   train loss 0.951058 acc 0.6206   test loss 0.532333 acc 0.7672
epoch 1   train loss 0.380394 acc 0.8631   test loss 0.329494 acc 0.8914
epoch 2   train loss 0.260320 acc 0.9106   test loss 0.293739 acc 0.8973
epoch 3   train loss 0.194880 acc 0.9343   test loss 0.311531 acc 0.8991
epoch 4   train loss 0.149834 acc 0.9487   test loss 0.329447 acc 0.8966
epoch 5   train loss 0.112276 acc 0.9618   test loss 0.394547 acc 0.8879
epoch 6   train loss 0.091287 acc 0.9718   test loss 0.423515 acc 0.8823
epoch 7   train loss 0.076412 acc 0.9765   test loss 0.473700 acc 0.8802
epoch 8   train loss 0.063544 acc 0.9803   test loss 0.513066 acc 0.8765
epoch 9   train loss 0.055481 acc 0.9832   test loss 0.556770 acc 0.8740
epoch 10   train loss 0.048097 acc 0.9846   test loss 0.559893 acc 0.8747
epoch 11   train loss 0.046436 acc 0.9867   test loss 0.575650 acc 0.8740
epoch 12   train loss 0.037619 acc 0.9886   test loss 0.613780 acc 0.8698
epoch 13   train loss 0.035444 acc 0.9889   test

In [ ]:
#TEST 14

model = SentenceModel(len(ds_full.vocab), 64, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.Adagrad(model.parameters(), lr=0.001) #lr: 0.1 -> 0.001
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 30)
# Testing converges at 60 epochs at 56% accuracy and 1.25 test loss

epoch 0   train loss 1.599456 acc 0.3277   test loss 1.574110 acc 0.3267
epoch 1   train loss 1.581549 acc 0.3285   test loss 1.562650 acc 0.3288
epoch 2   train loss 1.565260 acc 0.3289   test loss 1.544407 acc 0.3320
epoch 3   train loss 1.528551 acc 0.3299   test loss 1.504440 acc 0.3325
epoch 4   train loss 1.461355 acc 0.3335   test loss 1.446182 acc 0.3404
epoch 5   train loss 1.388492 acc 0.3652   test loss 1.398770 acc 0.3786
epoch 6   train loss 1.329965 acc 0.3988   test loss 1.374281 acc 0.3865
epoch 7   train loss 1.287208 acc 0.4097   test loss 1.361845 acc 0.3889
epoch 8   train loss 1.255134 acc 0.4168   test loss 1.347520 acc 0.3952
epoch 9   train loss 1.227670 acc 0.4244   test loss 1.338258 acc 0.3968
epoch 10   train loss 1.204755 acc 0.4294   test loss 1.334364 acc 0.4038
epoch 11   train loss 1.183898 acc 0.4411   test loss 1.324380 acc 0.4103
epoch 12   train loss 1.166505 acc 0.4495   test loss 1.317344 acc 0.4124
epoch 13   train loss 1.150294 acc 0.4603   test

## Example Outputs

Print the desired number of outputs using code below

In [ ]:
def print_outputs(correct_count=5, incorrect_count=5):
  _, _, preds, _ = run_test(model, ds_test, crit)

  # setup variables
  pred = []
  correct = []
  correct_prediction = []
  correct_actual = []
  incorrect = []
  incorrect_prediction = []
  incorrect_actual = []
  rand_corr_idx = []
  rand_incorr_idx = []

  # map results into appropriate arrays
  for i in range(len(preds)):
      pred.append(preds[i].item())                                              # transfer predictions from tensor to array

  for x in range(len(ds_test)):                                                 # compare every prediction with the actual sentiment, move the text to their respective arrays depending on result
    if pred[x] == ds_test[x][0]:
      correct.append(ds_test[x])                                                # correctly predicted senteces move to correct array
      correct_prediction.append(pred[x])                                        # also store prediction
      correct_actual.append(ds_test[x][0])                                      # place actual labels into correct_actual array
    else:
      incorrect.append(ds_test[x])                                              # same process as correct labels, but with incorrect predictions
      incorrect_prediction.append(pred[x])                                        
      incorrect_actual.append(ds_test[x][0])


  # choose random examples from results
  if (correct_count > len(correct)):                                            # make sure no index out of bounds
    correct_count = len(correct)  

  if (incorrect_count > len(incorrect)):
    incorrect_count = len(incorrect)

  for c in range(correct_count):                                                # pick random examples from correct arr
    index = random.randint(0,len(correct)-1)
    while (index in rand_corr_idx):                                             # make sure no duplicates
      index = random.randint(0,len(correct)-1)
    rand_corr_idx.append(index)

  for c in range(incorrect_count):                                              # pick random examples from incorrect arr
    index = random.randint(0,len(incorrect)-1)
    while (index in rand_incorr_idx):
      index = random.randint(0,len(incorrect)-1)
    rand_incorr_idx.append(index)

  # output results
  print("CORRECT PREDICTIONS:", len(correct), "\n")                             # print correct predictions, with their actual labels and sentence
  for y in range(correct_count):
    print("prediction: ", sentiment[correct_prediction[rand_corr_idx[y]]])  
    print("actual:     ", sentiment[correct_actual[rand_corr_idx[y]]])
    print("sentence:   ", ' '.join([ds_full.vocab.itos[x] for x in correct[rand_corr_idx[y]][1]]), "\n")

  print('===================================================================\n')

  print("INCORRECT PREDICTIONS:", len(incorrect), "\n")                         # print incorrect predictions, with their actual labels and sentence
  for z in range(incorrect_count):
    print("prediction: ", sentiment[incorrect_prediction[rand_incorr_idx[z]]])  
    print("actual:     ", sentiment[incorrect_actual[rand_incorr_idx[z]]])
    print("sentence:   ", ' '.join([ds_full.vocab.itos[x] for x in incorrect[rand_incorr_idx[z]][1]]), "\n")

In [ ]:
num_corr_out = 10
num_incorr_out = 10

print_outputs(num_corr_out, num_incorr_out)

CORRECT PREDICTIONS: 3904 

prediction:  anger
actual:      anger
sentence:    The more resentful and unforgiving Willie Morgan , was taken to court and then had to terminate a lucrative six year contract after only nine mon 

prediction:  anger
actual:      anger
sentence:    i feel frustrated that i ca nt answer questions for distributors or customers 

prediction:  happy
actual:      happy
sentence:    i feel privileged and beyond lucky to have met him 

prediction:  happy
actual:      happy
sentence:    i feel very energetic to cook something very special i decide to prepare at least one dish with posto and the other days when i simply do nt remain in the mood of cooking at all i again look for posto 

prediction:  sadness
actual:      sadness
sentence:    i have to keep fighting for my life until i truly run out of fight and i ve been close enough to that twice to know a bit about what it feels like and we re not there yet no matter how despairing all this feels 

prediction:  ang

## Conclusion

With the use of the embedding and LSTM model, I was able to achieve 91% accuracy. The hyperparameters listed below performed the best from my testing for the Emotions_final dataset. Due to the small amount of testing done so far, there is a high possibility there is a more optimized model for the respected dataset. From test 11 shown, the test loss increases over time, so decreasing test loss by lowering the learning rate or using fewer epochs will probably improve the model, which will be taken into consideration for future testing. Finally, looking at the mismatched results, some of the sentences and their corresponding labels are difficult for even a human to distinguished while other sentiments may not match the sentence either, so further cleaning of our dataset could be another strategy.

```
#TEST 11

model = SentenceModel(len(ds_full.vocab), 64, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=0.1)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)
```



# DO THIS FIRST IF YOU DON'T HAVE THE **DATASET**

Also you must first run most of the code above for the model and stuff to work.

In [58]:
!pip install -q kaggle
from google.colab import files 

In [ ]:
files.upload()
#here upload "kaggle.json"

In [63]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [66]:
!kaggle datasets download -d yamaerenay/spotify-dataset-19212020-160k-tracks

 93% 181M/195M [00:00<00:00, 207MB/s]
100% 195M/195M [00:00<00:00, 211MB/s]


In [ ]:
!unzip /content/spotify-dataset-19212020-160k-tracks.zip

# **DATASET CODE HERE**

upload -> sentence_model_state_dict (the saved fully trained model) 
---


In [11]:
model = SentenceModel(len(ds_full.vocab)+2, 64, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=0.1)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

In [12]:
# Model class must be defined somewhere
model.load_state_dict(torch.load("/content/sentence_model_state_dict.pth"))
model.eval()

SentenceModel(
  (embedding): Embedding(19910, 64)
  (lstm): LSTM(64, 64, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=64, out_features=6, bias=True)
)

In [14]:
ldr = torch.utils.data.DataLoader(ds_test) 
ldr.dataset[26][1]

tensor([    2,   129,    24,    98,   166,     2,    16,    10,   204,   668,
          522,   119,     2,   105,   139,     5, 11316,     5,  7635,    11,
         1094])

In [15]:
device = torch.device('cuda:0')
model.to(device)
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=0.1)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

with torch.no_grad():
    print('Tensor with ground truth label: {0}'.format(ldr.dataset[26]))
    out_incorrect = model(ldr.dataset[26][1].unsqueeze(0).to(device))
    print(out_incorrect)
    print("Predicted: {0}".format(out_incorrect.argmax(1)))
    print("Predicted correctly: {0}".format(out_incorrect.argmax(1) == ldr.dataset[26][0]))
    print("Tensor converted to text: " + ' '.join([ds_full.vocab.itos[x] for x in ldr.dataset[26][1]]))

Tensor with ground truth label: (4, tensor([    2,   129,    24,    98,   166,     2,    16,    10,   204,   668,
          522,   119,     2,   105,   139,     5, 11316,     5,  7635,    11,
         1094]))
tensor([[ -1.7010,  -2.3726,  -2.6075,   1.0321,  16.4197, -10.9642]],
       device='cuda:0')
Predicted: tensor([4], device='cuda:0')
Predicted correctly: tensor([True], device='cuda:0')
Tensor converted to text: i felt this way before i was feeling rather reluctant whether should i go down to bishan to fetch my boyfriend


In [16]:
def get_weights(tensor=None, model=None): #tensor should have shape ([x, y, z, ...]) (1 dim), NOT ([[x, y, z, ...]]), the function itself unsqueezes the input tensor for you
  tensor = tensor.unsqueeze(0)
  device = torch.device('cuda:0')
  model.to(device)
  m = nn.ReLU()
  s = nn.Softmax(dim=1)
  with torch.no_grad():
    update_tensor = model(tensor.to(device))
    relud_logged = torch.log(m(update_tensor))
    print(relud_logged)
    print("Predicted: {0}".format(update_tensor.argmax(1)))
    print("Tensor converted to text: " + ' '.join([ds_full.vocab.itos[x] for x in tensor.squeeze()]))
    return s(relud_logged).squeeze()
  
  # this function calculates the appropriate probability distribution for every track based on input "tensor" and our model

In [17]:
x = 100

print(ldr.dataset[x])
print("\n")
weight = get_weights(ldr.dataset[x][1], model)

(5, tensor([   2,   25,   10,   17, 4064,    4, 1330]))


tensor([[  -inf,   -inf, 1.3253,   -inf,   -inf, 2.1848]], device='cuda:0')
Predicted: tensor([5], device='cuda:0')
Tensor converted to text: i am feeling so hyper and bouncy


In [18]:
print(weight)
weight[2].item()

tensor([0.0000, 0.0000, 0.2974, 0.0000, 0.0000, 0.7026], device='cuda:0')


0.29743579030036926

In [19]:
tracks = pd.read_csv('/content/tracks.csv')

In [20]:
tracks.drop(inplace=True, columns=['duration_ms','key', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'tempo', 'time_signature', 'loudness'])

In [21]:
tracks.sort_values(by=['popularity'], ascending=False)

,id,name,popularity,explicit,artists,id_artists,release_date,danceability,energy,valence
93802,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),100.0,1.0,"['Justin Bieber', 'Daniel Caesar', 'Giveon']","['1uNFoZAHBGtllmzznpCI3s', '20wkVLutqVOYrc0kxF...",2021-03-19,0.677,0.6960,0.464
93803,7lPN2DXiMsVn7XUKtOW1CS,drivers license,99.0,1.0,['Olivia Rodrigo'],['1McMsnEElThX1knmY4oliG'],2021-01-08,0.585,0.4360,0.132
93804,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,98.0,0.0,['Masked Wolf'],['1uU7g3DNSbsu0QjSEqZtEd'],2021-01-06,0.778,0.6950,0.472
92810,5QO79kh1waicV47BqGRL3g,Save Your Tears,97.0,1.0,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2020-03-20,0.680,0.8260,0.644
92811,6tDDoYIxWvMLTdKpjFkc1B,telepatía,97.0,0.0,['Kali Uchis'],['1U1el3k54VvEUzo3ybLPlM'],2020-12-04,0.653,0.5240,0.553
...,...,...,...,...,...,...,...,...,...,...
122975,6BZ5YNaZ5ItGxq1uUG2A4h,"String Quartet No. 12 in E-Flat Major, Op. 127...",0.0,0.0,"['Ludwig van Beethoven', 'Pro Arte Quartet']","['2wOqMjp9TyABvtHdOSOTUS', '7LDDafwFTPDzeLj5mo...",1946,0.469,0.2260,0.659
122974,6BLfgzfqkQdNZJZsc4dhAr,The Christmas Song,0.0,0.0,['Zae Lee'],['3g5k4HKoosE7mLOu7GVVfv'],1946,0.517,0.0611,0.326
122973,6AhBrEvEprzbP9pSyLJgcZ,"Six épigraphes antiques, L. 131: II. Pour un t...",0.0,0.0,"['Claude Debussy', 'Robert Casadesus', 'Gaby C...","['1Uff91EOsvd99rtAupatMP', '60UJYXbYq8oGjl7FDq...",1946,0.307,0.0430,0.104
126088,1CgjNopMJ4nvS2O2DwaKzm,She Loves You,0.0,0.0,['The New Merseysiders'],['5oqAUfibbMIA08TufojASy'],2020-06-04,0.571,0.5600,0.935


sentiment = ['sadness', 'anger', 'love', 'surprise', 'fear', 'happy']

In [22]:
import numpy as np

In [23]:
# list_scores = []
# in = ('sadness', 'anger', 'love', 'surprise', 'fear', 'happy')
# for **track** in list of tracks over 65 popularity:
#    score = POPULARITY_WEIGHT * (track.popularity / 100) + in.sadness * (1 - track.valence) + in.anger * track.energy + in.love * (mean(track.energy + track.dancability)) + in.surprise * (1 - track.energy) + in.fear * valence + in.happy * (mean(track.danceability + track.energy))
#    list_scores.append(score)

# np.random.choice(tracks, list_scores)

In [24]:
new_df = tracks[tracks.popularity >=65]
new_df

,id,name,popularity,explicit,artists,id_artists,release_date,danceability,energy,valence
8095,1LGqJ3nvxpVXDWpEzq4DJD,All of Me,65.0,0.0,['Billie Holiday'],['1YzCsTRb22dQkh9lghPIrp'],1933,0.504,0.0644,0.403
22549,5NGtFXVpXSvwunEIGeviY3,Gymnopédie No. 1,69.0,0.0,"['Erik Satie', 'Philippe Entremont']","['459INk8vcC0ebEef82WjIK', '4x2fCu6MNErew0pb64...",1949,0.469,0.0128,0.354
27499,648TTtYB0bH0P8Hfy0FmkL,Unforgettable,68.0,0.0,['Nat King Cole'],['7v4imS0moSyGdXyLgVTIV7'],1954-01-01,0.349,0.1820,0.178
28498,6crfO56bDm0RjpctUuGs5X,I'm In The Mood For Love,65.0,0.0,['Julie London'],['3qUMmh5biaB5hqpF4LqS3m'],1955-12-01,0.580,0.0545,0.177
29498,3aEJMh1cXKEjgh52claxQp,I've Got You Under My Skin - Remastered 1998,68.0,0.0,['Frank Sinatra'],['1Mxqyy3pSjf8kZZL4QVxS0'],1956-03,0.585,0.2470,0.591
...,...,...,...,...,...,...,...,...,...,...
178903,25cUhiAod71TIQSNicOaW3,Adorn,67.0,0.0,['Miguel'],['360IAlyVv4PCEVjgyMZrxK'],2012-09-28,0.625,0.5760,0.235
178952,63CSozvYUEudPp12679UVF,Don't Worry (with Ray Dalton) - Radio Edit,69.0,0.0,"['Madcon', 'Ray Dalton']","['6c4sUNBgdonFJz8Kx2VsGz', '4e0nWw2r4BoQSKPQ2z...",2015-04-10,0.811,0.6990,0.733
178955,5ubvP9oKmxLUVq506fgLhk,Stayin Alive,79.0,0.0,['Bee Gees'],['1LZEQNv7sE11VDY3SdxQeN'],2020-11-20,0.702,0.7730,0.953
178956,2cX2coZS1PYBfPs8wgbdWE,"More Than A Woman - From ""Saturday Night Fever...",79.0,0.0,['Bee Gees'],['1LZEQNv7sE11VDY3SdxQeN'],2020-11-20,0.601,0.7030,0.673


In [25]:
def normalize(p): # makes sure the distribution probability list adds to 1
    if sum(p) != 1.0:
        p = np.asarray(p)*(1.0/sum(p))
    return p

In [50]:
#user input is x -> we have to convert user input string into tensor "x"
#so input would be "input = get_weights(x, model)"

POPULARITY_WEIGHT = 2 # can be changed 
NUM_TRACK_OPTIONS = 5 # can be changed

id_tracks = new_df["id"].values.tolist()
scores_id = []
input = get_weights(ldr.dataset[x][1], model)
for track in new_df.itertuples(index=False):
    score = POPULARITY_WEIGHT*(track.popularity/100) + input[0].item()*(1-track.valence) + input[1].item()*track.energy + input[2].item()*((track.energy + track.danceability)/2) + input[3].item()*(1-track.energy) + input[4].item()*track.valence + input[5].item()*((track.danceability + track.energy)/2)
    scores_id.append([score, track.id])

distribution = []
top_track_ids = []
s = sorted(scores_id, reverse=True)
s = s[0:NUM_TRACK_OPTIONS]

for score, id in s:
  distribution.append(score)
  top_track_ids.append(id)
print("\n")
id = np.random.choice(top_track_ids, p=normalize(distribution)) # picks a random track across a distribution generated by the scores
print("Distribution: {0}".format(input))
print("TRACK LINK: https://open.spotify.com/track/" + id)
actual_track = new_df[new_df['id']==id]
print(actual_track)

tensor([[  -inf,   -inf, 1.3253,   -inf,   -inf, 2.1848]], device='cuda:0')
Predicted: tensor([5], device='cuda:0')
Tensor converted to text: i am feeling so hyper and bouncy


Distribution: tensor([0.0000, 0.0000, 0.2974, 0.0000, 0.0000, 0.7026], device='cuda:0')
TRACK LINK: https://open.spotify.com/track/4iJyoBOLtHqaGxP12qzhQI
                           id  ... valence
93802  4iJyoBOLtHqaGxP12qzhQI  ...   0.464

[1 rows x 10 columns]
